#### <span style="color:red">Note: this notebook requires Python3 kernel</span>

#### Init user specific variables

In [1]:
import os
user='maibendayan' # 'hagailevi'# 'maibendayan'
os.chdir(f'/specific/elkon/{user}/PRS/codebase')
user_prefix="mai_"

In [2]:
from constants import *
import pandas as pd

#### Init analysis vars

In [3]:
# Datasets
discoveries=[f"{user_prefix}bcac_onco_afr_no_1200_usa-5pcs", f"{user_prefix}bcac_onco_eur-5pcs"] # ["bcac_onco_eas1-5pcs", "bcac_onco_eur-5pcs"] # ["bcac_onco_afr_no_1200_usa-5pcs", "bcac_onco_eur-5pcs"]
### make sure to sort the discoveries lexicographically! ###
mono_discovery0=discoveries[0]
mono_discovery1=discoveries[1]
multi_discovery="+".join(discoveries)
gwas_pops=["EAS", "EUR"] 
target= f"{user_prefix}bcac_onco_afr_1200_usa" # "bcac_onco_eas2"
imp="impX"
pheno="pheno"

# Method
method="ls"

# Cross validation parameters
n_folds=3
n_reps=4
base_rep=103 # 102 # n_repetitions+99

# Multi PRS file suffices
inner_test_name='validation.multi'
test_name='test.multi'

# Other file suffices
metric='all.auroc' # 'or.all'
rank_by='or'

Concatenate all reps files into a single DF (and optinally saving the results)

In [4]:
def create_outer_stat_file(discovery,target,imp,test_name,n_reps,n_folds, base_rep, method,save_file=False, rank_by=""):
    
    metric_suffix=""
    if rank_by:
        metric_suffix=f'_by_{rank_by}'

    mono_outer_all_df=pd.DataFrame()
    for cur_rep in range(1, n_reps+1):
        prs_rep_path=f'{PRSS_PATH}{discovery}_{target}/{imp}/rep_{str(base_rep)}_{str(cur_rep)}/prs.cv.{method}___'
        file_name=prs_rep_path+str(n_folds)+'_'+test_name+'.statistics'+metric_suffix+'.tsv'
        df_stat=pd.read_csv(file_name, sep='\t')
        # df_stat.set_index('hp', inplace = True)
        df_stat['rep']=cur_rep
        df_stat[f'{metric}.outer.rank']=len(df_stat[metric])-df_stat[metric].rank()+1
        
        # sum the results
        mono_outer_all_df=pd.concat([mono_outer_all_df,df_stat])

    if save_file:
        output_path=PRSS_PATH+discovery+'_'+target+'/'+imp+'/prs.cv.'+method+'___'
        mono_outer_all_df.to_csv(output_path+'all_'+str(n_reps)+'_'+test_name+'.statistics'+param_suffix+'.tsv',mode="w", sep='\t')
        print(mono_outer_all_df)
        print ("the next file was created: "+output_path+'all_'+str(n_reps)+'_'+test_name+'.statistics'+param_suffix+'.tsv')

    return mono_outer_all_df

In [5]:
# test_name='test'
# create_outer_stat_file(discoveries,target,imp,test_name,n_reps,n_folds,base_rep, method,save_file=False)

Average of statistics results w/o using the multi step (mono-ethnic only)

In [6]:
# Do not forget to run this first!
test_name='test'

In [7]:
# non-EUR
df_non_eur_all=create_outer_stat_file(discoveries[0],target,imp,test_name,n_reps,n_folds,base_rep, method,save_file=False) 
df_non_eur_all_agg=df_non_eur_all[['hp', metric, f'{metric}.outer.rank', 'rep']].groupby('hp')[[metric]].mean()

FileNotFoundError: [Errno 2] No such file or directory: '/specific/elkon/maibendayan/PRS/PRSs/mai_bcac_onco_afr_no_1200_usa-5pcs_mai_bcac_onco_afr_1200_usa/impX/rep_103_1/prs.cv.ls___3_test.statistics.tsv'

In [ ]:
# EUR
df_eur_all=create_outer_stat_file(discoveries[1],target,imp,test_name,n_reps,n_folds,base_rep, method,save_file=False)
df_eur_all_agg=df_eur_all[['hp', metric, f'{metric}.outer.rank', 'rep']].groupby('hp')[[metric]].mean()

Average of statistics results including the multi step

In [ ]:
# Do not forget to run this first!
multi_discovery='+'.join(discoveries)
test_name='test.multi'

In [ ]:
df_multi_all=create_outer_stat_file(multi_discovery,target,imp,test_name,n_reps,n_folds,base_rep, method,save_file=False, rank_by='or')
df_multi_all_agg=df_multi_all[['hp', metric, f'{metric}.outer.rank', 'rep']].groupby('hp')[[metric]].mean()

#### Generic function for aggregating all folds by reps and hps
the function returns the following dataframes:
- The performance obtained on each rep according to the optimal **inner** hps - this is the 
- The performance obtained on each rep according to the optimal **outer** hps - along with extra metadata

In [ ]:
def prs_analysis(discovery,target,imp,outer_test_name,inner_test_name,n_folds,reps,base_rep,method,rank_by=""):

    df_outer_all = create_outer_stat_file(discovery,target,imp,outer_test_name,reps,n_folds,base_rep, method)
    
    metric_suffix=""
#     fold_range=list(range(1,n_folds+1))
    if rank_by:
        metric_suffix=f'_by_{rank_by}'
#         fold_range=['avg']
        
    # df of statistics for inner loops over all reps:
    df_cat=pd.DataFrame()
    dfs=[]
    for rep in range(1,reps+1):
        for fold in range(1,n_folds+1):
            path=PRSS_PATH+discovery+'_'+target+'/'+imp+'/rep_'+str(base_rep)+'_'+str(rep)+'/'+'prs.cv.'+method+'___'
            path=f'{path}{fold}_{n_folds}_{inner_test_name}.statistics{metric_suffix}.tsv'
            inner_df=pd.read_csv(path, sep='\t')
            inner_df['fold']=fold
            inner_df['rep']=rep
            dfs.append(inner_df)
    
    df_cat=pd.concat(dfs, ignore_index=True)

#     # Extract optimal hps for outer CV
#     df_outer_optimal_hps=df_cat.loc[df_cat.groupby('rep')['or.all'].idxmax(),['rep', 'fold', 'hp']]
#     display(df_outer_optimal_hps)
    
    # Check te performance of the optimal inner hps on the outer CV
    df_inner_all=pd.concat(dfs, ignore_index=True)
    df_inner_mean_or=df_inner_all.groupby(['rep', 'hp'])[[metric]].mean().reset_index()
    df_optimal_hps=df_inner_mean_or.loc[df_inner_mean_or.groupby('rep')[metric].idxmax()]

    # Check te performance of the optimal outer hps on the outer CV
    df_outer_all.index=df_outer_all.apply(lambda a: f'{a["rep"]}_{a["hp"]}', axis=1)
    print(df_outer_all.index)
    result_true=df_outer_all.loc[df_optimal_hps.apply(lambda a: f'{a["rep"]}_{a["hp"]}', axis=1)]
#     print('the or.all in the outer loop using the optimal hps in the inner loop: \n(best practice)')
#     display(result_true['or.all'])
    
#     print('the max or.all in the outer loop')
#     display(df_outer_all.groupby('rep')[['or.all']].max())

    
    max_or_all = df_outer_all.groupby('rep')[metric].max()
    result_max = pd.merge(df_outer_all, max_or_all, on=['rep', metric])
# #     print('and their full info')
# #     display(result_max)

    return result_true, result_max

# prs_analysis(mono_discovery0,target,imp,'test','validation',n_folds,reps,base_rep, method, rank_by)

In [ ]:
# non-EUR
df_true, df_max=prs_analysis(mono_discovery0,target,imp,'test','validation',n_folds,n_reps,base_rep, method, rank_by)
df_non_eur_outer_rep=df_true[['hp', metric, f'{metric}.outer.rank', 'rep']]
display(df_non_eur_outer_rep)
print("mean:", round(df_true[metric].mean(),4))

In [ ]:
# EUR
df_true, df_max=prs_analysis(mono_discovery1,target,imp,'test','validation',n_folds,n_reps,base_rep,method, rank_by)
df_eur_outer_rep=df_true[['hp', metric, f'{metric}.outer.rank', 'rep']]
display(df_eur_outer_rep)
print("mean:", round(df_true[metric].mean(),4))

In [ ]:
# MULTI
rank_by=""
df_true, df_max=prs_analysis(multi_discovery,target,imp,'test.multi','validation.multi',n_folds,n_reps,base_rep,method)
display(df_true[['hp', metric, f'{metric}.outer.rank', 'rep']]) # .groupby('hp')[['or.all']].mean()
print("mean:", round(df_true[metric].mean(),4))

In [ ]:
# MULTI2

def extract_top_mono_hp_sets(a):
    for i, (cur_non_eur, cur_eur) in enumerate(zip(df_non_eur_outer_rep['hp'], df_eur_outer_rep['hp'])):
        if a['hp1']==cur_non_eur and a['hp2']==cur_eur and a['rep']==i+1:
            return True
    return False

hp_series=df_multi_all['hp']
hp_type=(float if method.startswith('pt') else str)
df_multi_all['hp1']=hp_series.apply(lambda a : a.split('+')[0]).astype(hp_type).values
df_multi_all['hp2']=hp_series.apply(lambda a : a.split('+')[1]).astype(hp_type).values
df_multi_from_mono=df_multi_all[df_multi_all.apply(extract_top_mono_hp_sets, axis=1)]
display(df_multi_from_mono[['hp', metric, f'{metric}.outer.rank', 'rep']])
df_multi_from_mono[metric].mean()
